In [ ]:
# Sample Code to demonstrate how to use the Batch Prediction API to score data living in an Azure Blob Storage container and output the results back to the same container

In [ ]:
import datarobot as dr

In [ ]:
# Set DataRobot connection info here
API_KEY ='YOUR API KEY'
BATCH_PREDICTIONS_URL = "https://app.datarobot.com/api/v2"
DEPLOYMENT_ID = 'YOUR DEPLOYMENT ID'

# Set Azure connection blob info here
AZURE_STORAGE_ACCOUNT = "YOUR AZURE STORAGE ACCOUNT NAME"
AZURE_STORAGE_CONTAINER = "YOUR AZURE STORAGE ACCOUNT CONTAINER"
# you can get the storage access key by going to "Access keys" in the left menu of the storage account and use either key
AZURE_STORAGE_ACCESS_KEY = "AZURE STORAGE ACCOUNT ACCESS KEY"

# Set Azure input and output file names 
AZURE_INPUT_SCORING_FILE = "YOUR INPUT SCORING FILE NAME"
AZURE_OUTPUT_RESULTS_FILE = "YOUR OUTPUT RESULTS FILE NAME"

# Set name for azure credential in DataRobot
DR_CREDENTIAL_NAME = "Azure_{}".format(AZURE_STORAGE_ACCOUNT)


In [ ]:
dr.Client(token=API_KEY,endpoint=BATCH_PREDICTIONS_URL)

In [ ]:
# Create an Azure-specific Credential 
# Connection String format: DefaultEndpointsProtocol=https;AccountName=****;AccountKey=****
# The connection string is also found below the access key in Azure if you want to copy that directly.

credential = dr.Credential.create_azure(
    name=DR_CREDENTIAL_NAME,
    azure_connection_string="DefaultEndpointsProtocol=https;AccountName={};AccountKey={};".format(AZURE_STORAGE_ACCOUNT, AZURE_STORAGE_ACCESS_KEY)
)

credential

In [ ]:
# Use this code to look up the ID of the credential object created.
credential_id = None
for cred in dr.Credential.list():
    if cred.name == DR_CREDENTIAL_NAME:
        credential_id = cred.credential_id

print(credential_id)

In [ ]:
# Set up our batch prediction job
# Input: Azure Blob Storage
# Output: Azure Blob Storage

job = dr.BatchPredictionJob.score(
    deployment=DEPLOYMENT_ID,
    intake_settings={
        'type': 'azure',
        'url': "https://{}.blob.core.windows.net/{}/{}".format(AZURE_STORAGE_ACCOUNT, AZURE_STORAGE_CONTAINER,AZURE_INPUT_SCORING_FILE),
        "credential_id": credential_id
    },
    output_settings={
        'type': 'azure',
        'url': "https://{}.blob.core.windows.net/{}/{}".format(AZURE_STORAGE_ACCOUNT, AZURE_STORAGE_CONTAINER,AZURE_OUTPUT_RESULTS_FILE),
        "credential_id": credential_id
    },
    # If explanations are required, uncomment the line below
    max_explanations=5,

    # If passthrough columns are required, use this line
    passthrough_columns=['column1','column2']
)


In [ ]:
job.wait_for_completion()
job.get_status()